<a href="https://colab.research.google.com/github/akshatshah91/Game-AI/blob/master/GA_with_NN_for_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install pyvirtualdisplay
#!pip install pyglet==v1.3.2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  libgle3
The following NEW packages will be installed:
  python-opengl
0 upgraded, 1 newly installed, 0 to remove and 11 not upgraded.
Need to get 496 kB of archives.
After this operation, 5,416 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-opengl all 3.1.0+dfsg-1 [496 kB]
Fetched 496 kB in 0s (6,095 kB/s)
Selecting previously unselected package python-opengl.
(Reading database ... 144676 files and directories currently installed.)
Preparing to unpack .../python-opengl_3.1.0+dfsg-1_all.deb ...
Unpacking python-opengl (3.1.0+dfsg-1) ...
Setting up python-opengl (3.1.0+dfsg-1) ...
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 11 not upgraded.
Need to get 783 kB o

In [41]:
import numpy as np
import matplotlib.pyplot as plt
import gym
import torch
import torch.nn as nn
from torch.autograd import Variable

In [78]:
input_size     = 4     # Cart pos and speed
hidden_size    = 10    # The number of nodes at the hidden layer
output_size    = 2     # Number of outputs: left and right
generations    = 100
startingSize   = 100
bestPopulation = 10
averageRuns    = 5

In [79]:
class CartPoleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()                    # Inherited from the parent class nn.Module
        self.fc1 = nn.Linear(input_size, hidden_size)  # 1st Full-Connected Layer: 784 (input data) -> 500 (hidden node)
        self.relu = nn.ReLU()                          # Non-Linear ReLU Layer: max(0,x)
        self.fc2 = nn.Linear(hidden_size, output_size) # 2nd Full-Connected Layer: 500 (hidden node) -> 10 (output class)
    
    def forward(self, x):                              # Forward pass: stacking each layer together
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [52]:
def init_weights(w):
  if(type(w)==nn.Linear):
    print("yes")
    torch.nn.init.xavier_uniform(w.weight)
    w.bias.data.fill(0.00)

In [81]:
def generatePopulation(popSize):
  agents = []
  for x in range(popSize):
    tmp = CartPoleNN(input_size, hidden_size, output_size)
    init_weights(tmp)
    agents.append(tmp)
  return agents

In [ ]:
def selectBest(agents, num):
  rewards = []
  

In [74]:
env = gym.make("CartPole-v1")
net = Net(input_size, hidden_size, num_classes)
for param in net.parameters():
    param.requires_grad = False
init_weights(net)
action = 0
for x in range(5):
  g = env.reset()
  reward = 0
  gameOver = False
  while not gameOver:
    output = net(Variable(torch.from_numpy(g).type("torch.FloatTensor")))
    if output[0] > output[1]:
      action = 0
    else:
      action = 1
    g, r, gameOver, _ = env.step(action)
    reward += r
  print(reward)

no
9.0
8.0
10.0
10.0
10.0


In [76]:
parents = generatePopulation(startingSize)
for gen in range(generations):
  selected = selectBest(parents, bestPopulation)

SyntaxError: ignored